In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**import lib**

In [ ]:
import math
import os

import sys

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
from keras.preprocessing.image import array_to_img
from PIL import Image as im 


## import data

In [ ]:
train_data=pd.read_csv("../input/img-dim-train/train_img_dim.csv")
train_data.head()

In [ ]:
annots=train_data.copy()
annots= annots[annots.class_id != 14] #drop NO Finding class
annots.shape

In [ ]:
len(set(annots.image_id))

In [ ]:
listfile=[]
files=os.listdir("../input/vinbigdata-chest-xray-abnormalities-detection/train") 
for file in files: 
    if os.path.isfile("../input/vinbigdata-chest-xray-abnormalities-detection/train/" + file):
        listfile.append(file)
len(listfile)

## norm bbox to new images

In [ ]:
IMG_SIZE=512
def norm_df(df):
    df.x_min= ((df.x_min/ df.width)* IMG_SIZE).astype('int32')
    df.x_max=( (df.x_max/ df.width)* IMG_SIZE).astype('int32')
    
    df.y_min= ((df.y_min/ df.height)* IMG_SIZE).astype('int32')
    df.y_max= ((df.y_max/ df.height)* IMG_SIZE).astype('int32')
    return df
df=norm_df(annots.copy()).reset_index(drop = True)

df.to_csv('all_data_dim_convert.csv')

## make folder 

In [ ]:
# if you want
'''
DATA_DIR = "../input/vinbigdata-chest-xray-abnormalities-detection"
train_dcm_dir = os.path.join(DATA_DIR, "train")
test_dcm_dir = os.path.join(DATA_DIR, "test")

img_dir = os.path.join(os.getcwd(), "images")  # .jpg
csv_dir = os.path.join(os.getcwd(), "df")  # .csv

for directory in [img_dir, csv_dir]:
    if os.path.isdir(directory):
        continue
    os.mkdir(directory)

'''    

## All unique picture is 4394


In [ ]:

mylist=[]
for image_id in df.image_id.unique():
    mylist.append(image_id)
print('all file is : ' ,len(mylist))
    

## convert dicom to jpg

## Now we convert dicom to JPG or PNG  thanks to [raddar](http://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way)

In [ ]:

i=0
for item in mylist:
    image_id=item
    voi_lut = True
    fix_monochrome = True
    dicom = pydicom.dcmread('../input/vinbigdata-chest-xray-abnormalities-detection/train/'+image_id+'.dicom')
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
 
    #X_train11 =data_file.pixel_array
    new_array = cv2.resize(data, (IMG_SIZE,IMG_SIZE))# norm the value dont div on 255 
    new_array = np.reshape(new_array, (IMG_SIZE, IMG_SIZE,1))
    new_array = np.asarray(new_array)
    img = array_to_img(new_array)
    i=i+1
    if (i+1)%500==0:
        print(i+1)

    img.save('/kaggle/working/' + image_id + '.jpg',dtype=np.uint16, quality=100)


### makeclasses csv file

In [ ]:

'''
classes=df.class_name.unique()
class_id=df.class_id.unique()
print(class_id)
print(classes)

with open('classes.csv', mode='w') as file:
    for  clss ,i in zip(classes,class_id ):
        file.write('{},{}\n'.format(clss, i))
cass=pd.read_csv('classes.csv')
cass
'''


### add path to images in  csv

In [ ]:
'''

df.image_id= df.image_id.apply(lambda x: '/kaggle/working/images'+ '/' + x + '.jpg')
df_path=df.copy()
df_path.head()


'''
